In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

from keras_preprocessing.image import DataFrameIterator, ImageDataGenerator
import random
import numpy as np
import pandas as pd
from keras.layers import Dense, Flatten, Input, Reshape
from keras.models import Sequential, Model
from PIL import Image

Using TensorFlow backend.


### Base dataframe

In [2]:
pixel_val = 1
np_filenames, img_filenames = [], []
for i in range(20):
    np_filename = '/tmp/{}.npy'.format(i)
    img_filename = '/tmp/{}.jpg'.format(i)
    array = pixel_val * np.random.randint(0, 255, size=(2, 2)).astype('float32')
    np.save(np_filename, array[..., None])
    plt.imsave(img_filename, array)
    np_filenames.append(np_filename)
    img_filenames.append(img_filename)
    
df = pd.DataFrame({'img_path': img_filenames, 'np_filename': np_filenames}).sample(frac=1).reset_index(drop=True)
df['img_path_extra'] = img_filenames
df['regression'] = np.random.randint(1, 10, size=len(df))
df['regression_extra'] = np.random.randint(10, 100, size=len(df))
df['binary'] = random.sample(['dog', 'cat'] * 30, len(df))
df['multi_class'] = random.sample(['dog', 'cat', 'horse'] * 30, len(df))
df['multi_label'] = random.sample(['dog', 'cat', ['dog'], ['cat'], ['cat', 'dog'], ['dog', 'cat']] * 10, len(df))
print(len(df))
df.head()

20


,img_path,np_filename,img_path_extra,regression,regression_extra,binary,multi_class,multi_label
0,/tmp/8.jpg,/tmp/8.npy,/tmp/0.jpg,4,72,cat,cat,"[cat, dog]"
1,/tmp/13.jpg,/tmp/13.npy,/tmp/1.jpg,8,71,dog,cat,dog
2,/tmp/6.jpg,/tmp/6.npy,/tmp/2.jpg,7,87,dog,dog,"[dog, cat]"
3,/tmp/0.jpg,/tmp/0.npy,/tmp/3.jpg,8,62,dog,horse,[dog]
4,/tmp/12.jpg,/tmp/12.npy,/tmp/4.jpg,4,65,dog,cat,[cat]


### Inference

In [3]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    batch_size=2
)
print(df_iter.inputs)
print(df_iter.outputs)
for x in next(df_iter):
    print(x.shape)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]
[]
(2, 255, 255, 3)


In [4]:
df_iter = DataFrameIterator(
    df, 
    input_columns=['img_path'],
    batch_size=2
)
print(df_iter.inputs)
print(df_iter.outputs)
for x in next(df_iter):
    print(x.shape)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]
[]
(2, 255, 255, 3)


In [5]:
df_iter = DataFrameIterator(
    df, 
    input_columns=['img_path', 'img_path_extra'],
    batch_size=2
)
print(df_iter.inputs)
print()
print(df_iter.outputs)
for x in next(df_iter):
    print(x.shape)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}, {'column': 'img_path_extra', 'values': ['/tmp/0.jpg', '/tmp/1.jpg', '/tmp/2.jpg', '/tmp/3.jpg', '/tmp/4.jpg', '/tmp/5.jpg', '/tmp/6.jpg', '/tmp/7.jpg', '/tmp/8.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/13.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/16.jpg', '/tmp/17.jpg', '/tmp/18.jpg', '/tmp/19.jpg']}]

[]
(2, 255, 255, 3)
(2, 255, 255, 3)


### Regression

In [6]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns='regression',
    batch_size=2
)
print(df_iter.inputs)
print()
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(255, 255, 3)))
model.add(Dense(1))
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'regression', 'mode': None, 'values': [4, 8, 7, 8, 4, 9, 2, 6, 4, 8, 4, 2, 9, 4, 9, 3, 6, 3, 6, 4]}]


W1008 08:31:37.429927 140055512885056 deprecation_wrapper.py:119] From /home/rodrigo/anaconda3/envs/keras-dev/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/1
10/10 [==============================] - 0s 13ms/step - loss: 100815074.0377


In [7]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['regression'],
    batch_size=2
)
print(df_iter.inputs)
print()
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(1))
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'values': ['/tmp/8.npy', '/tmp/13.npy', '/tmp/6.npy', '/tmp/0.npy', '/tmp/12.npy', '/tmp/18.npy', '/tmp/14.npy', '/tmp/15.npy', '/tmp/3.npy', '/tmp/11.npy', '/tmp/16.npy', '/tmp/10.npy', '/tmp/4.npy', '/tmp/1.npy', '/tmp/5.npy', '/tmp/9.npy', '/tmp/2.npy', '/tmp/19.npy', '/tmp/7.npy', '/tmp/17.npy']}]

[{'column': 'regression', 'mode': None, 'values': [4, 8, 7, 8, 4, 9, 2, 6, 4, 8, 4, 2, 9, 4, 9, 3, 6, 3, 6, 4]}]
Epoch 1/1
10/10 [==============================] - 0s 10ms/step - loss: 133433180.0582


### Binary classification

In [8]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

W1008 08:31:38.126500 140055512885056 deprecation.py:323] From /home/rodrigo/anaconda3/envs/keras-dev/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'binary', 'mode': 'sparse', 'values': [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 12ms/step - loss: 4362.8710


In [9]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    color_mode='grayscale',
    image_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(2,2,1)))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'values': ['/tmp/8.npy', '/tmp/13.npy', '/tmp/6.npy', '/tmp/0.npy', '/tmp/12.npy', '/tmp/18.npy', '/tmp/14.npy', '/tmp/15.npy', '/tmp/3.npy', '/tmp/11.npy', '/tmp/16.npy', '/tmp/10.npy', '/tmp/4.npy', '/tmp/1.npy', '/tmp/5.npy', '/tmp/9.npy', '/tmp/2.npy', '/tmp/19.npy', '/tmp/7.npy', '/tmp/17.npy']}]

[{'column': 'binary', 'mode': 'sparse', 'values': [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 9ms/step - loss: 41.8529


### Multi-class classsification

In [43]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'sparse'},
    batch_size=2,
    color_mode='grayscale',
    image_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(2,2,1)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'sparse_categorical_crossentropy')
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'multi_class', 'mode': 'sparse', 'values': [0, 0, 1, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0], 'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 38ms/step - loss: 191.8427


In [44]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'sparse'},
    batch_size=2,
    color_mode='grayscale',
    image_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(2,2,1)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'sparse_categorical_crossentropy')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'values': ['/tmp/8.npy', '/tmp/13.npy', '/tmp/6.npy', '/tmp/0.npy', '/tmp/12.npy', '/tmp/18.npy', '/tmp/14.npy', '/tmp/15.npy', '/tmp/3.npy', '/tmp/11.npy', '/tmp/16.npy', '/tmp/10.npy', '/tmp/4.npy', '/tmp/1.npy', '/tmp/5.npy', '/tmp/9.npy', '/tmp/2.npy', '/tmp/19.npy', '/tmp/7.npy', '/tmp/17.npy']}]

[{'column': 'multi_class', 'mode': 'sparse', 'values': [0, 0, 1, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0], 'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 39ms/step - loss: 108.7028


In [45]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'categorical_crossentropy')
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'multi_class', 'mode': 'categorical', 'values': [], 'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1


IndexError: list index out of range

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'categorical_crossentropy')
model.fit_generator(df_iter)

### Multi-label classification

In [40]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_label'],
    output_modes={'multi_label': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(2, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'multi_label', 'mode': 'categorical', 'values': [[0, 1], [1, 0], [1], [0], [1, 0], [1, 0], [1, 0], [1, 0], [0], [0], [0], [0, 1], [0]], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
 1/10 [==>...........................] - ETA: 3s - loss: 165.5659

IndexError: list index out of range

In [32]:
list(np.array([12]))

[12]

In [15]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['multi_label'],
    output_modes={'multi_label': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(2, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'values': ['/tmp/8.npy', '/tmp/13.npy', '/tmp/6.npy', '/tmp/0.npy', '/tmp/12.npy', '/tmp/18.npy', '/tmp/14.npy', '/tmp/15.npy', '/tmp/3.npy', '/tmp/11.npy', '/tmp/16.npy', '/tmp/10.npy', '/tmp/4.npy', '/tmp/1.npy', '/tmp/5.npy', '/tmp/9.npy', '/tmp/2.npy', '/tmp/19.npy', '/tmp/7.npy', '/tmp/17.npy']}]

[{'column': 'multi_label', 'mode': 'categorical', 'values': [array([0, 1]), 1, array([1, 0]), array([1]), array([0]), 0, 0, array([1, 0]), array([1, 0]), 0, array([1, 0]), 0, 1, array([1, 0]), array([0]), 1, array([0]), array([0]), array([0, 1]), array([0])], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 20ms/step - loss: 8025.3385


### Output same as input

In [16]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['img_path'],
    output_modes={'img_path': 'image'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

x, y = next(df_iter)
assert np.allclose(x[0], y[0])
assert id(x[0]) != id(y[0])

model = Sequential()
model.add(Flatten(input_shape=(255, 255, 3)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(255 * 255 * 3, activation='sigmoid'))
model.add(Reshape((255, 255, 3)))
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'img_path', 'mode': 'image', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]
Epoch 1/1
10/10 [==============================] - 5s 538ms/step - loss: 17163.8450


### Segmentation

In [17]:
df_iter = DataFrameIterator(
    df,
    input_columns='img_path',
    output_columns='img_path_extra',
    output_modes={'img_path_extra': 'image'},
    batch_size=1,
    shuffle=False,
    image_data_generator=ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
)

print(df_iter.inputs)
print()
print(df_iter.outputs)
print()

x, y = next(df_iter)
print(len(x[0]))
print(x[0][0].shape)
print(len(y[0]))
print(y[0][0].shape)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'img_path_extra', 'mode': 'image', 'values': ['/tmp/0.jpg', '/tmp/1.jpg', '/tmp/2.jpg', '/tmp/3.jpg', '/tmp/4.jpg', '/tmp/5.jpg', '/tmp/6.jpg', '/tmp/7.jpg', '/tmp/8.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/13.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/16.jpg', '/tmp/17.jpg', '/tmp/18.jpg', '/tmp/19.jpg']}]

1
(255, 255, 3)
1
(255, 255, 3)


### Multi-output: regression  & regression

In [26]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'regression_extra'],
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1)(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'regression', 'mode': None, 'values': [4, 8, 7, 8, 4, 9, 2, 6, 4, 8, 4, 2, 9, 4, 9, 3, 6, 3, 6, 4]}, {'column': 'regression_extra', 'mode': None, 'values': [72, 71, 87, 62, 65, 55, 66, 78, 37, 44, 51, 30, 47, 77, 55, 85, 93, 47, 30, 29]}]
Epoch 1/1
10/10 [==============================] - 0s 27ms/step - loss: 313645044.5023 - dense_29_loss: 156927872.0000 - dense_30_loss: 156717184.0000


In [25]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['regression', 'regression_extra'],
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1)(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'values': ['/tmp/8.npy', '/tmp/13.npy', '/tmp/6.npy', '/tmp/0.npy', '/tmp/12.npy', '/tmp/18.npy', '/tmp/14.npy', '/tmp/15.npy', '/tmp/3.npy', '/tmp/11.npy', '/tmp/16.npy', '/tmp/10.npy', '/tmp/4.npy', '/tmp/1.npy', '/tmp/5.npy', '/tmp/9.npy', '/tmp/2.npy', '/tmp/19.npy', '/tmp/7.npy', '/tmp/17.npy']}]

[{'column': 'regression', 'mode': None, 'values': [4, 8, 7, 8, 4, 9, 2, 6, 4, 8, 4, 2, 9, 4, 9, 3, 6, 3, 6, 4]}, {'column': 'regression_extra', 'mode': None, 'values': [72, 71, 87, 62, 65, 55, 66, 78, 37, 44, 51, 30, 47, 77, 55, 85, 93, 47, 30, 29]}]
Epoch 1/1
10/10 [==============================] - 0s 22ms/step - loss: 484266665.3313 - dense_27_loss: 319720768.0000 - dense_28_loss: 164545952.0000


### Multi-output: regression & binary classification

In [24]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'binary_crossentropy'])
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'regression', 'mode': None, 'values': [4, 8, 7, 8, 4, 9, 2, 6, 4, 8, 4, 2, 9, 4, 9, 3, 6, 3, 6, 4]}, {'column': 'binary', 'mode': 'sparse', 'values': [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 24ms/step - loss: 150380487.6922 - dense_25_loss: 150375152.0000 - dense_26_loss: 5338.6436


### Multi-output: regression & multi-class classification

In [23]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['regression', 'multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(3, activation='softmax')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'categorical_crossentropy'])
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]

[{'column': 'regression', 'mode': None, 'values': [4, 8, 7, 8, 4, 9, 2, 6, 4, 8, 4, 2, 9, 4, 9, 3, 6, 3, 6, 4]}, {'column': 'multi_class', 'mode': 'categorical', 'values': [0, 0, 1, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0], 'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 27ms/step - loss: 148878645.9227 - dense_23_loss: 148874784.0000 - dense_24_loss: 3873.3372


### Multi-output: multi-class & multi-label classification

In [22]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class', 'multi_label'],
    output_modes={'multi_class': 'categorical', 'multi_label': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(255, 255, 3))
x = Flatten()(_input)
out_1 = Dense(3, activation='softmax')(x)
out_2 = Dense(2, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['categorical_crossentropy', 'binary_crossentropy'])
model.fit_generator(df_iter)

[{'column': 'img_path', 'values': ['/tmp/8.jpg', '/tmp/13.jpg', '/tmp/6.jpg', '/tmp/0.jpg', '/tmp/12.jpg', '/tmp/18.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/3.jpg', '/tmp/11.jpg', '/tmp/16.jpg', '/tmp/10.jpg', '/tmp/4.jpg', '/tmp/1.jpg', '/tmp/5.jpg', '/tmp/9.jpg', '/tmp/2.jpg', '/tmp/19.jpg', '/tmp/7.jpg', '/tmp/17.jpg']}]
[{'column': 'multi_class', 'mode': 'categorical', 'values': [0, 0, 1, 2, 0, 0, 2, 0, 2, 1, 1, 0, 2, 0, 0, 0, 2, 1, 1, 0], 'index_to_class': {0: 'cat', 1: 'dog', 2: 'horse'}}, {'column': 'multi_label', 'mode': 'categorical', 'values': [array([0, 1]), 1, array([1, 0]), array([1]), array([0]), 0, 0, array([1, 0]), array([1, 0]), 0, array([1, 0]), 0, 1, array([1, 0]), array([0]), 1, array([0]), array([0]), array([0, 1]), array([0])], 'index_to_class': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 33ms/step - loss: 26430.0147 - dense_21_loss: 19891.2090 - dense_22_loss: 6538.8037
